In [2]:
from sklearn.cluster import KMeans
import pandas as pd
import re
import warnings

warnings.filterwarnings('ignore')

In [4]:
# Load the dataset from a CSV file
df = pd.read_csv('X_train_G3tdtEn.csv')

# Get the data types of all columns
dtypes = df.dtypes

cols = df.columns

for col in cols:
    if "goods_code" in col or "model" in col or "ID" in col or col == 'Nb_of_items':
        df.drop(col, axis=1, inplace=True)

cols = df.columns

df.fillna('', inplace=True)

df.head()

,item1,item2,item3,item4,item5,item6,item7,item8,item9,item10,...,Nbr_of_prod_purchas15,Nbr_of_prod_purchas16,Nbr_of_prod_purchas17,Nbr_of_prod_purchas18,Nbr_of_prod_purchas19,Nbr_of_prod_purchas20,Nbr_of_prod_purchas21,Nbr_of_prod_purchas22,Nbr_of_prod_purchas23,Nbr_of_prod_purchas24
0,COMPUTERS,,,,,,,,,,...,,,,,,,,,,
1,COMPUTER PERIPHERALS ACCESSORIES,,,,,,,,,,...,,,,,,,,,,
2,TELEVISIONS HOME CINEMA,,,,,,,,,,...,,,,,,,,,,
3,COMPUTERS,COMPUTER PERIPHERALS ACCESSORIES,,,,,,,,,...,,,,,,,,,,
4,TELEVISIONS HOME CINEMA,,,,,,,,,,...,,,,,,,,,,


In [5]:
unique_values = {}
feature_cols = {}
feature_df = {}
features = ['item', 'price', 'make', 'purchas']

for feature in features:
    unique_values[feature] = []
    feature_cols[feature] = []
    
    for col in cols:
        if feature in col:
            feature_cols[feature].append(col)
    
    feature_df[feature] = df[feature_cols[feature]]
        
    if feature == 'item' or feature == 'make':  
        
        feature_df[feature] = feature_df[feature].apply(lambda x: x.str.replace(' ', ''))
        
        for i, row in feature_df[feature].iterrows():
            for val in row:
                if val != '':
                    if val not in unique_values[feature]:
                        unique_values[feature].append(val)
                        
print(unique_values)

{'item': ['COMPUTERS', 'COMPUTERPERIPHERALSACCESSORIES', 'TELEVISIONSHOMECINEMA', 'SERVICE', 'FULFILMENTCHARGE', 'CABLESADAPTERS', 'BEDROOMFURNITURE', 'LIVING&DININGFURNITURE', 'LIVINGDININGFURNITURE', 'WARRANTY', 'COMPUTERPERIPHERALS&ACCESSORIES', 'TELEPHONES,FAXMACHINES&TWO-WAYRADIOS', 'BABY&CHILDTRAVEL', 'TELEPHONESFAXMACHINESTWO-WAYRADIOS', 'STANDSBRACKETS', 'HOMEOFFICE', 'TELEVISIONS&HOMECINEMA', 'LIGHTING', 'OUTDOORFURNITURE', 'SOFTFURNISHINGS', 'AUDIOACCESSORIES', 'BATHROOMACCESSORIES', 'TABLEWARE', 'WINDOWDRESSING', 'BARBECUES&ACCESSORIES', 'OUTDOORACCESSORIES', 'POWERBATTERIES', 'BEDLINEN', 'BABYCHILDTRAVEL', 'HEALTHBEAUTYELECTRICAL', 'BAGSCARRYCASES', 'HOUSEHOLDCLEANING', 'BARBECUESACCESSORIES', 'FOODPREPARATION', 'COOKWARE', 'HI-FI', 'CABLES&ADAPTERS', 'CARPETSRUGSFLOORING', 'STORAGEORGANISATION', 'BATHLINEN', 'KITCHENUTENSILSGADGETS', 'DECORATIVEACCESSORIES', 'DECORATING', 'POWER&BATTERIES', 'HEATING&COOLINGAPPLIANCES', 'KITCHENSTORAGE', 'LAUNDRYCLOTHESCARE', 'BABYFEEDING',

In [6]:
new_cols = []

for feature in features:
    if feature == 'item' or feature == 'make':       
        for label_add_up in [' Number', ' Price']:
            for val in unique_values[feature]:
                new_cols.append(val + label_add_up)

new_cols

['COMPUTERS Number',
 'COMPUTERPERIPHERALSACCESSORIES Number',
 'TELEVISIONSHOMECINEMA Number',
 'SERVICE Number',
 'FULFILMENTCHARGE Number',
 'CABLESADAPTERS Number',
 'BEDROOMFURNITURE Number',
 'LIVING&DININGFURNITURE Number',
 'LIVINGDININGFURNITURE Number',
 'WARRANTY Number',
 'COMPUTERPERIPHERALS&ACCESSORIES Number',
 'TELEPHONES,FAXMACHINES&TWO-WAYRADIOS Number',
 'BABY&CHILDTRAVEL Number',
 'TELEPHONESFAXMACHINESTWO-WAYRADIOS Number',
 'STANDSBRACKETS Number',
 'HOMEOFFICE Number',
 'TELEVISIONS&HOMECINEMA Number',
 'LIGHTING Number',
 'OUTDOORFURNITURE Number',
 'SOFTFURNISHINGS Number',
 'AUDIOACCESSORIES Number',
 'BATHROOMACCESSORIES Number',
 'TABLEWARE Number',
 'WINDOWDRESSING Number',
 'BARBECUES&ACCESSORIES Number',
 'OUTDOORACCESSORIES Number',
 'POWERBATTERIES Number',
 'BEDLINEN Number',
 'BABYCHILDTRAVEL Number',
 'HEALTHBEAUTYELECTRICAL Number',
 'BAGSCARRYCASES Number',
 'HOUSEHOLDCLEANING Number',
 'BARBECUESACCESSORIES Number',
 'FOODPREPARATION Number',
 'CO

In [11]:
count_item = {}
count_make = {}
price_item = {}
price_make = {}

for i, row in feature_df['item'].iterrows():
    count_item[i] = {}
    price_item[i] = {}
    
    for item in unique_values['item']:
        count_item[i][f'{item} Number'] = 0
        price_item[i][f'{item} Price'] = 0
    
    for index, val in row.iteritems():
        if val != '':
            i_th = re.sub("[^0-9]", "", index)
            count_item[i][f'{val} Number'] += float(feature_df['purchas'][f'Nbr_of_prod_purchas{i_th}'][i])
            price_item[i][f'{val} Price'] += float(feature_df['price'][f'cash_price{i_th}'][i])

In [14]:
for i, row in feature_df['make'].iterrows():
    count_make[i] = {}
    price_make[i] = {}
    
    for make in unique_values['make']:
        count_make[i][f'{make} Number'] = 0
        price_make[i][f'{make} Price'] = 0
    
    for index, val in row.iteritems():
        if val != '':
            i_th = re.sub("[^0-9]", "", index)
            count_make[i][f'{val} Number'] += float(feature_df['purchas'][f'Nbr_of_prod_purchas{i_th}'][i])
            price_make[i][f'{val} Price'] += float(feature_df['price'][f'cash_price{i_th}'][i])
            

In [15]:
df_count_item = pd.DataFrame.from_dict(count_item, orient='index')

df_count_item.head()

,COMPUTERS Number,COMPUTERPERIPHERALSACCESSORIES Number,TELEVISIONSHOMECINEMA Number,SERVICE Number,FULFILMENTCHARGE Number,CABLESADAPTERS Number,BEDROOMFURNITURE Number,LIVING&DININGFURNITURE Number,LIVINGDININGFURNITURE Number,WARRANTY Number,...,2HPELITEBOOK850V6 Number,"2MICROSOFTOFFICEHOMEANDSTUDENT2019, Number",2TARGUSGEOLITEESSENTIALCASE Number,2LOGITECHPEBBLEM350BLUETOOTHMOUSE Number,APPLES Number,BLANKMEDIA&MEDIASTORAGE Number,PRODUCT Number,TECHNOLOGYACCESSORIES Number,HOMESAFETYEQUIPMENT Number,HEALTHBEAUTYACCESSORIES Number
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df_price_item = pd.DataFrame.from_dict(price_item, orient='index')

df_price_item.head()

,COMPUTERS Price,COMPUTERPERIPHERALSACCESSORIES Price,TELEVISIONSHOMECINEMA Price,SERVICE Price,FULFILMENTCHARGE Price,CABLESADAPTERS Price,BEDROOMFURNITURE Price,LIVING&DININGFURNITURE Price,LIVINGDININGFURNITURE Price,WARRANTY Price,...,2HPELITEBOOK850V6 Price,"2MICROSOFTOFFICEHOMEANDSTUDENT2019, Price",2TARGUSGEOLITEESSENTIALCASE Price,2LOGITECHPEBBLEM350BLUETOOTHMOUSE Price,APPLES Price,BLANKMEDIA&MEDIASTORAGE Price,PRODUCT Price,TECHNOLOGYACCESSORIES Price,HOMESAFETYEQUIPMENT Price,HEALTHBEAUTYACCESSORIES Price
0,889.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,409.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1399.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,689.0,119.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1199.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_count_make = pd.DataFrame.from_dict(count_make, orient='index')

df_count_make.head()

,APPLE Number,SAMSUNG Number,SONY Number,RETAILER Number,LG Number,BUGABOO Number,PANASONIC Number,SWYFT Number,HERMANMILLER Number,HYPNOS Number,...,LOLAROSE Number,BONDS Number,LEAPFROG Number,MADEBYZEN Number,EUFY Number,GEORGJENSEN Number,MINIMODERNS Number,FLORENCEBROADHURST Number,ALADDIN Number,TATE Number
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_price_make = pd.DataFrame.from_dict(price_make, orient='index')

df_price_make.head()

,APPLE Price,SAMSUNG Price,SONY Price,RETAILER Price,LG Price,BUGABOO Price,PANASONIC Price,SWYFT Price,HERMANMILLER Price,HYPNOS Price,...,LOLAROSE Price,BONDS Price,LEAPFROG Price,MADEBYZEN Price,EUFY Price,GEORGJENSEN Price,MINIMODERNS Price,FLORENCEBROADHURST Price,ALADDIN Price,TATE Price
0,889.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,409.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1399.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,808.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1199.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
data = pd.concat([df_count_item, df_price_item, df_count_make, df_price_make], axis=1)

data.head()

,COMPUTERS Number,COMPUTERPERIPHERALSACCESSORIES Number,TELEVISIONSHOMECINEMA Number,SERVICE Number,FULFILMENTCHARGE Number,CABLESADAPTERS Number,BEDROOMFURNITURE Number,LIVING&DININGFURNITURE Number,LIVINGDININGFURNITURE Number,WARRANTY Number,...,LOLAROSE Price,BONDS Price,LEAPFROG Price,MADEBYZEN Price,EUFY Price,GEORGJENSEN Price,MINIMODERNS Price,FLORENCEBROADHURST Price,ALADDIN Price,TATE Price
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
